<a href="https://colab.research.google.com/github/sudarshan-koirala/youtube-stuffs/blob/main/langchain/synthetic_data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### How Is it done ??
### Before GenAI emerged (Well, this is used still in many companies but **GenAI is what everyone is talking about these days** )
Some of the modelling techniques used:
- Classic statistical methods
- Deep Learning models (GAN , VAE behind the scene)
- Mix of classic statistical models and Deep Learning.

Once synthetic data is generated, we need to evaluate it to make sure it is OK to use in downstream tasks. There are many libraries, websites offering these kind of solutions. But here, we focus in GenAI part.

## Use case

Synthetic data refers to artificially generated data that imitates the characteristics of real data without containing any information from actual individuals or entities. It is typically created through mathematical models, algorithms, or other data generation techniques. Synthetic data can be used for a variety of purposes, including testing, research, and training machine learning models, while preserving privacy and security.

Benefits of Synthetic Data:

1. **Privacy and Security**: No real personal data at risk of breaches.
2. **Data Augmentation**: Expands datasets for machine learning.
3. **Flexibility**: Create specific or rare scenarios.
4. **Cost-effective**: Often cheaper than real-world data collection.
5. **Regulatory Compliance**: Helps navigate strict data protection laws.
6. **Model Robustness**: Can lead to better generalizing AI models.
7. **Rapid Prototyping**: Enables quick testing without real data.
8. **Controlled Experimentation**: Simulate specific conditions.
9. **Access to Data**: Alternative when real data isn't available.

**Note: Despite the benefits, synthetic data should be used carefully, as it may not always capture real-world complexities.**

## Quickstart

In this notebook, we'll dive deep into generating synthetic medical billing records using the langchain library. This tool is particularly useful when you want to develop or test algorithms but don't want to use real patient data due to privacy concerns or data availability issues.

## Setup
- First, you'll need to have the langchain library installed, along with its dependencies. Since we're using the OpenAI generator chain, we'll install that as well. Since this is an experimental lib, we'll need to include `langchain_experimental` in our installs.
- [Pydantic](https://docs.pydantic.dev/latest/): Data validation library for Python

In [1]:
%%capture
# !pip install -U langchain langchain_experimental openai

In [2]:
# set environment variables
# https://platform.openai.com/account/api-keys
# import os
# os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [3]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.pydantic_v1 import BaseModel
from langchain_experimental.tabular_synthetic_data.base import SyntheticDataGenerator
from langchain_experimental.tabular_synthetic_data.openai import create_openai_data_generator, OPENAI_TEMPLATE
from langchain_experimental.tabular_synthetic_data.prompts import SYNTHETIC_FEW_SHOT_SUFFIX, SYNTHETIC_FEW_SHOT_PREFIX

## 1. Define Your Data Model
- Every dataset has a structure or a "schema".
- The MedicalBilling class below serves as our schema for the synthetic data.
- By defining this, we're informing our synthetic data generator about the shape and nature of data we expect.

In [4]:
class MedicalBilling(BaseModel):
    patient_id: int(unique=True)
    patient_name: str
    diagnosis_code: str
    procedure_code: str
    total_charge: float
    insurance_claim_amount: float


## 2. Sample Data
To guide the synthetic data generator, it's useful to provide it with a few real-world-like examples. These examples serve as a "seed" - they're representative of the kind of data you want, and the generator will use them to create more data that looks similar.

Here are some fictional medical billing records:

In [5]:
examples = [
    {"example": """Patient ID: 123456, Patient Name: John Doe, Diagnosis Code:
        J20.9, Procedure Code: 99203, Total Charge: $500, Insurance Claim Amount: $350"""},
    {"example": """Patient ID: 789012, Patient Name: Johnson Smith, Diagnosis
        Code: M54.5, Procedure Code: 99213, Total Charge: $150, Insurance Claim Amount: $120"""},
    {"example": """Patient ID: 345678, Patient Name: Emily Stone, Diagnosis Code:
        E11.9, Procedure Code: 99214, Total Charge: $300, Insurance Claim Amount: $250"""},
]

## 3. Craft a Prompt Template
The generator doesn't magically know how to create our data; we need to guide it. We do this by creating a prompt template. This template helps instruct the underlying language model on how to produce synthetic data in the desired format.

In [6]:
OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)

The `FewShotPromptTemplate` includes:

- `prefix` and `suffix`: These likely contain guiding context or instructions.
- `examples`: The sample data we defined earlier.
- `input_variables`: These variables ("subject", "extra") are placeholders you can dynamically fill later. For instance, "subject" might be filled with "medical_billing" to guide the model further.
- `example_prompt`: This prompt template is the format we want each example row to take in our prompt.

## 4. Creating the Data Generator
With the schema and the prompt ready, the next step is to create the data generator. This object knows how to communicate with the underlying language model to get synthetic data.

In [7]:
synthetic_data_generator = create_openai_data_generator(
    output_schema=MedicalBilling,
    llm=ChatOpenAI(temperature=1),
    prompt=prompt_template,
)

c:\Users\jdamodhar\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## 5. Generate Synthetic Data
Finally, let's get our synthetic data!

In [8]:
synthetic_results = synthetic_data_generator.generate(
    subject="medical_billing",
    extra="the name must be chosen at random. Make it something you wouldn't normally choose.",
    runs=100,
)

This command asks the generator to produce 10 synthetic medical billing records. The results are stored in `synthetic_results`. The output will be a list of the MedicalBilling pydantic models.

In [9]:
type(synthetic_results)

list

## 6. Visualize the Generated Synthetic Data

In [10]:
len(synthetic_results)

100

In [11]:
synthetic_results

[MedicalBilling(patient_id=123456, patient_name='Alice Johnson', diagnosis_code='G40.909', procedure_code='99203', total_charge=600.0, insurance_claim_amount=400.0),
 MedicalBilling(patient_id=456789, patient_name='Samantha Rodriguez', diagnosis_code='I10', procedure_code='99212', total_charge=200.0, insurance_claim_amount=150.0),
 MedicalBilling(patient_id=987654, patient_name='Xavier Thompson', diagnosis_code='M25.511', procedure_code='99213', total_charge=400.0, insurance_claim_amount=300.0),
 MedicalBilling(patient_id=999999, patient_name="Evangeline O'Malley", diagnosis_code='S72.502A', procedure_code='99204', total_charge=800.0, insurance_claim_amount=600.0),
 MedicalBilling(patient_id=123456, patient_name='Finnegan McLeod', diagnosis_code='E78.5', procedure_code='99203', total_charge=300.0, insurance_claim_amount=250.0),
 MedicalBilling(patient_id=456789, patient_name='Seraphina Rodriguez', diagnosis_code='G47.33', procedure_code='99214', total_charge=600.0, insurance_claim_amou

## 7. Converting the synthetic data into Pandas Dataframe

In [12]:
import pandas as pd

# Create a list of dictionaries from the objects
synthetic_data = []
for item in synthetic_results:
    synthetic_data.append({
        'patient_id': item.patient_id,
        'patient_name': item.patient_name,
        'diagnosis_code': item.diagnosis_code,
        'procedure_code': item.procedure_code,
        'total_charge': item.total_charge,
        'insurance_claim_amount': item.insurance_claim_amount
    })

# Create a Pandas DataFrame from the list of dictionaries
synthetic_df = pd.DataFrame(synthetic_data)

# Display the DataFrame
print(type(synthetic_df))
synthetic_df

<class 'pandas.core.frame.DataFrame'>


,patient_id,patient_name,diagnosis_code,procedure_code,total_charge,insurance_claim_amount
0,123456,Alice Johnson,G40.909,99203,600.0,400.0
1,456789,Samantha Rodriguez,I10,99212,200.0,150.0
2,987654,Xavier Thompson,M25.511,99213,400.0,300.0
3,999999,Evangeline O'Malley,S72.502A,99204,800.0,600.0
4,123456,Finnegan McLeod,E78.5,99203,300.0,250.0
...,...,...,...,...,...,...
95,13579,Prudence Bumbleberry,E78.5,58558,2000.0,1300.0
96,54321,Zephyr Thunderclaw,N39.0,41212,2200.0,1400.0
97,24680,Juno Fizzlepop,R67.0,12345,1800.0,1200.0
98,98765,Rutherford McSnortle,I10,78901,2500.0,1600.0


In [13]:
synthetic_df.shape

(100, 6)

### Start exploring based on your usecase and use the same approach for real sensitive data. But, be careful, as the synthetic data might not capture the real-world complexities.